In [69]:
import random

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Subset
from torchvision import models
from torchvision.models import resnet101
from tqdm import tqdm

In [70]:
cinic_directory = 'C:/Users/Dell/.cache/kagglehub/datasets/mengcius/cinic10/versions/1'
cinic_mean = [0.47889522, 0.47227842, 0.43047404]
cinic_std = [0.24205776, 0.23828046, 0.25874835]

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=cinic_mean,std=cinic_std)
])

transform_augment_advanced = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=cinic_mean,std=cinic_std),
    transforms.RandomErasing(p=0.5, scale=(0.02, 0.2), ratio=(0.3, 3.3), value=0)
])


cinic_train = DataLoader(
    torchvision.datasets.ImageFolder(cinic_directory + '/train', transform=transform),
    batch_size=128, shuffle=True)
cinic_valid = DataLoader(
    torchvision.datasets.ImageFolder(cinic_directory + '/valid', transform=transform),
    batch_size=128, shuffle=False)

In [71]:
def train(model, device, cinic_train, num_epochs = 10, lr=0.0001):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=0.001)

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        train_loader_tqdm = tqdm(cinic_train, desc=f"Epoka {epoch+1}/{num_epochs}")

        for images, labels in train_loader_tqdm:
            images, labels = images.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
            train_loader_tqdm.set_postfix(loss=loss.item())

        avg_loss = running_loss / len(cinic_train)
        accuracy = 100 * correct / total
        print(f"Epoka [{epoch+1}/{num_epochs}], Strata: {avg_loss:.4f}, Dokładność: {accuracy:.2f}%")

    print("Trenowanie zakończone!")

In [72]:
def test(model, device, cinic_valid):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in cinic_valid:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f"Dokładność na zbiorze walidacyjnym: {100 * correct / total:.2f}%")

In [73]:
def get_few_shot_subset(dataset, num_classes=10, shots_per_class=90):
    targets = np.array(dataset.targets)
    
    unique_classes = np.unique(targets)
    selected_classes = random.sample(list(unique_classes), num_classes)
    
    few_shot_indices = []
    for c in selected_classes:
        class_indices = np.where(targets == c)[0]
        sampled_indices = random.sample(class_indices.tolist(), min(shots_per_class, len(class_indices)))
        few_shot_indices.extend(sampled_indices)
    
    return Subset(dataset, few_shot_indices)

cinic_train_dataset = torchvision.datasets.ImageFolder(cinic_directory + '/train', transform=transform)
few_shot_train = get_few_shot_subset(cinic_train_dataset, num_classes=10, shots_per_class=10)

few_shot_loader = DataLoader(few_shot_train, batch_size=10, shuffle=True)

In [77]:
class FewShotModel(nn.Module):
    def __init__(self, backbone=resnet101(weights=models.ResNet101_Weights.IMAGENET1K_V1), num_classes=10):
        super().__init__()
        self.encoder = nn.Sequential(*list(backbone.children())[:-1])
        self.fc = nn.Linear(2048, num_classes)

    def forward(self, x):
        x = self.encoder(x)
        x = torch.flatten(x, start_dim=1)
        return self.fc(x)

In [78]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")
model = FewShotModel(num_classes=10).to(device)

In [79]:
train(model, device, few_shot_loader, num_epochs=10)

Epoka 1/10: 100%|██████████| 10/10 [00:03<00:00,  2.94it/s, loss=2.26]


Epoka [1/10], Strata: 2.4241, Dokładność: 11.00%


Epoka 2/10: 100%|██████████| 10/10 [00:03<00:00,  3.05it/s, loss=1.96]


Epoka [2/10], Strata: 1.9891, Dokładność: 31.00%


Epoka 3/10: 100%|██████████| 10/10 [00:03<00:00,  2.94it/s, loss=1.53]


...

Epoka [8/10], Strata: 0.6064, Dokładność: 86.00%


Epoka 9/10: 100%|██████████| 10/10 [00:03<00:00,  2.77it/s, loss=0.744]


Epoka [9/10], Strata: 0.6338, Dokładność: 84.00%


Epoka 10/10: 100%|██████████| 10/10 [00:03<00:00,  2.92it/s, loss=0.26]

Epoka [10/10], Strata: 0.4639, Dokładność: 93.00%
Trenowanie zakończone!


In [80]:
test(model, device, cinic_valid)

Dokładność na zbiorze walidacyjnym: 24.45%
